In [1]:
import numpy as np
import pandas as pd

In [2]:
product_types = {'Small': 'CM00', 'Large': 'CM10', 'Espresso':'EM00'}
models = {'Base':'1', 'Premium':'2', 'Deluxe':'3'}
pods = {'Small':'CP0', 'Large':'CP1', 'Expresso':'EP0'}
flavors = {'vanilla':'0', 'caramel':'1', 'psl':'2', 'mocha':'3', 'hazelnut':'4'}
packages = [1, 3, 5, 7]
index_names = ['Flavor', 'Package']
sku_list = ['CP001','CP101','CP003','CP103','EP003','EP005','EP007','CP011','CP111','CP013','CP113','EP013','EP015',
            'EP017','CP021','CP121','CP023','CP123','CP031','CP131','CP033','CP133','CP041','CP141','CP043','CP143']

In [3]:
def check_item(pod, flavor, package):
    sku = pods[pod] + flavors[flavor] + str(package)
    if sku in sku_list:
        return sku

def cross_sell():
    pod_list = []
    for flavor in flavors:
        for package in packages:
            for pod in pods:
                pod_list.append(check_item(pod, flavor, package))
    return pod_list

In [4]:
def coffee_machine_dataframe(product_types, models):   
    products = []
    for i in product_types:
        for j in models:
            products.append(product_types[i] + models[j])         
    data = np.array(products).reshape(len(product_types), len(models))   
    return pd.DataFrame(data=data, index=list(product_types.keys()), columns=list(models.keys())).transpose()

def pods_dataframe(models, flavors, packages):
    
    # Index Levels
    outside = np.repeat(list(flavors.keys()), len(packages))
    inside = packages*len(flavors)
    hier_index = list(zip(outside,inside))
    hier_index = pd.MultiIndex.from_tuples(hier_index)
    
    # generate dataframe
    data = np.array(cross_sell()).reshape(len(flavors)*len(packages), len(models))
    df = pd.DataFrame(data=data, index=hier_index, columns=list(product_types.keys()))
    df.index.names = index_names
    return df

In [5]:
coffee_machines = coffee_machine_dataframe(product_types, models)
coffee_machines

,Small,Large,Espresso
Base,CM001,CM101,EM001
Premium,CM002,CM102,EM002
Deluxe,CM003,CM103,EM003


In [6]:
coffee_pods = pods_dataframe(models, flavors, packages)
coffee_pods

Small  Large Espresso
Flavor   Package                       
vanilla  1        CP001  CP101     None
         3        CP003  CP103    EP003
         5         None   None    EP005
         7         None   None    EP007
caramel  1        CP011  CP111     None
         3        CP013  CP113    EP013
         5         None   None    EP015
         7         None   None    EP017
psl      1        CP021  CP121     None
         3        CP023  CP123     None
         5         None   None     None
         7         None   None     None
mocha    1        CP031  CP131     None
         3        CP033  CP133     None
         5         None   None     None
         7         None   None     None
hazelnut 1        CP041  CP141     None
         3        CP043  CP143     None
         5         None   None     None
         7         None   None     None

## Questions

In [7]:
# All large machines
coffee_machines['Large']

Base       CM101
Premium    CM102
Deluxe     CM103
Name: Large, dtype: object

In [8]:
# Cross-sell for all large machines
coffee_pods['Large'].dropna()

Flavor    Package
vanilla   1          CP101
          3          CP103
caramel   1          CP111
          3          CP113
psl       1          CP121
          3          CP123
mocha     1          CP131
          3          CP133
hazelnut  1          CP141
          3          CP143
Name: Large, dtype: object

In [9]:
# All cross-sell for large machine, smallest per flavor
[coffee_pods.Large[key][coffee_pods.Large[key].first_valid_index()] for key, value in flavors.items()]

['CP101', 'CP111', 'CP121', 'CP131', 'CP141']

In [10]:
# All choices on espresso vanilla landing page
coffee_pods.Espresso.loc['vanilla'].dropna()

Package
3    EP003
5    EP005
7    EP007
Name: Espresso, dtype: object

In [11]:
# All espresso machines
coffee_machines.Espresso

Base       EM001
Premium    EM002
Deluxe     EM003
Name: Espresso, dtype: object

In [12]:
# Cross-sell for all espresso machines
coffee_pods['Espresso'].dropna()

Flavor   Package
vanilla  3          EP003
         5          EP005
         7          EP007
caramel  3          EP013
         5          EP015
         7          EP017
Name: Espresso, dtype: object

In [13]:
# All cross-sell for espresso machine, smallest per flavor
cross_list = []
for key, value in flavors.items():
    try:
        cross_list.append(coffee_pods.Espresso[key][coffee_pods.Espresso[key].first_valid_index()])
    except KeyError:
        break
cross_list

['EP003', 'EP013']

In [14]:
# All choices on general vanilla landing page, smallest per product type
[coffee_pods.loc['vanilla'][key][coffee_pods.loc['vanilla'][key].first_valid_index()] \
     for key, value in product_types.items()]

['CP001', 'CP101', 'EP003']

In [16]:
# save data frames for the api
coffee_machines.to_csv('machines.csv', encoding='utf-8')
coffee_pods.to_csv('pods.csv', encoding='utf-8')